In [1]:
from GraphTsetlinMachine.graphs import Graphs
import numpy as np
from scipy.sparse import csr_matrix
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine
from time import time
import argparse
import random
import csv

In [2]:
n_clauses = 20000
# input_file = 'datasets/5x5_Simulated.csv'
input_file = 'datasets/11x11-4.csv'
dim = 11

def default_args(**kwargs):
    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", default=450, type=int)
    parser.add_argument("--number-of-clauses", default=n_clauses, type=int)
    parser.add_argument("--T", default=int(n_clauses / 1.6), type=int)
    parser.add_argument("--s", default=1, type=float)
    parser.add_argument("--depth", default=1, type=int)
    parser.add_argument("--hypervector-size", default=128, type=int)
    parser.add_argument("--hypervector-bits", default=2, type=int)
    parser.add_argument("--message-size", default=1024, type=int)
    parser.add_argument("--message-bits", default=2, type=int)
    parser.add_argument('--double-hashing', dest='double_hashing', default=False, action='store_true')
    parser.add_argument("--max-included-literals", default=32, type=int)

    args, unknown = parser.parse_known_args()
    for key, value in kwargs.items():
        if key in args.__dict__:
            setattr(args, key, value)
    return args

args = default_args()

In [3]:
print("Loading data")

with open(input_file, mode='r', newline='') as infile:
    reader = csv.reader(infile)
    data = list(reader)

random.shuffle(data)

# Remove parts of data for faster training, #Disabled
delete_index = 0 #int(0.9 * len(data))
data = data[delete_index:]

split_index = int(0.5 * len(data))
train_data = data[:split_index]
test_data = data[split_index:]

X_train = np.array([row[:-1] for row in train_data], dtype=int)
Y_train = np.array([row[-1:] for row in train_data], dtype=int)

# Separate X_data and Y_data for testing set
X_test = np.array([row[:-1] for row in test_data], dtype=int)
Y_test = np.array([row[-1:] for row in test_data], dtype=int)

# Make it to the correct dimension
X_train = X_train.reshape(-1, dim, dim)
X_test = X_test.reshape(-1, dim, dim)

Y_train = Y_train.reshape(-1)
Y_test = Y_test.reshape(-1)

# Change -1 to 0 so the output is binary (0 or 1)
Y_train = np.where(Y_train == -1, 0, Y_train)
Y_test = np.where(Y_test == -1, 0, Y_test)

# print("X_train shape:", X_train.shape)
# print("Y_train shape:", Y_train.shape)
# print("X_test shape:", X_test.shape)
# print("Y_test shape:", Y_test.shape)


Loading data


In [4]:
# patch_size = 1
# map_size = dim - patch_size + 1
map_size = dim

number_of_nodes = map_size * map_size

# print(patch_size, map_size, number_of_nodes)
print(map_size, number_of_nodes)

print("Creating training data")
s1 = [f"x:{i}" for i in range(0, dim)]
s2 = [f"y:{i}" for i in range(0, dim)]
symbols = ['A', 'B', 'O']

directions = [
    (-1, 0), (1, 0), 
    (0, -1), (0, 1), 
    (-1, 1), (1, -1)  
]
direction_labels = {
    (-1, 0): "left",
    (1, 0): "right",
    (0, -1): "up",
    (0, 1): "down",
    (-1, 1): "up_left",
    (1, -1): "down_right"
}

# for c in range(7):
#     symbols.append(f"neighbors_A:{c}")
#     symbols.append(f"neighbors_B:{c}")
#     symbols.append(f"neighbors_O:{c}")

symbols.append("is_corner:True")
symbols.append("is_corner:False")
symbols.append("is_border:True")
symbols.append("is_border:False")

# Add symbols for if the position is even or odd, # Testing feature, not sure how much will come from this
# symbols.append("parity_x:even")
# symbols.append("parity_x:odd")
# symbols.append("parity_y:even")
# symbols.append("parity_y:odd")

# Add direction for neighbors
for dir_lbl in direction_labels.values():
    for brick_type in ['A', 'B', 'O']:
        symbols.append(f"{dir_lbl}:{brick_type}")
    # If you want to encode "no neighbor" as well, you can do:
    symbols.append(f"{dir_lbl}:None")

symbols = [*symbols, *s1, *s2]

# Distance from center (maximum for a 5x5 is about 4 if the center is (2,2))
# for dist in range(11):
#     symbols.append(f"dist_center:{dist}")

# print(len(symbols))

center_x = map_size // 2 
center_y = map_size // 2


#### Helper functions ####
def manhattan_dist_to_center(x, y):
    return abs(x - center_x) + abs(y - center_y)

def is_corner(x, y, grid_size):
    # For a 5x5 grid, corners are (0,0), (0,4), (4,0), (4,4).
    return (x == 0 and y == 0) or \
           (x == grid_size-1 and y == 0) or \
           (x == 0 and y == grid_size-1) or \
           (x == grid_size-1 and y == grid_size-1)

def is_border(x, y, grid_size):
    # Node is on border if x or y is 0, or x or y is grid_size-1
    return (x == 0 or y == 0 or x == grid_size-1 or y == grid_size-1)

def parity_label(value):
    # Checks if even or odd, # ended up not using this
    return "even" if (value % 2 == 0) else "odd"

11 121
Creating training data


In [5]:
# Setup graph for training
graphs_train = Graphs(
    X_train.shape[0],
    symbols=symbols,
    hypervector_size=args.hypervector_size,
    hypervector_bits=args.hypervector_bits,
)


for graph_id in range(X_train.shape[0]):
    graphs_train.set_number_of_graph_nodes(graph_id, number_of_nodes)
    
graphs_train.prepare_node_configuration()
print("Finished preparing node configuration")


# Add the nodes
for graph_id in range(X_train.shape[0]):
    for x in range(map_size):
        for y in range(map_size):
            node_id = x * map_size + y
            number_of_outgoing_edges = 0
            
            for dx, dy in directions:
                neighbor_x = x + dx
                neighbor_y = y + dy
                
                # Check if neighbor exists within bounds
                if 0 <= neighbor_x < map_size and 0 <= neighbor_y < map_size:
                    number_of_outgoing_edges += 1
            # Use the node_counter to create a unique label for each node
            graphs_train.add_graph_node(graph_id, node_id, number_of_outgoing_edges)
            
    
print("All nodes are acounted for")
graphs_train.prepare_edge_configuration()
edge_type = "Plain"


# Add edges
print("Started adding edges")
for graph_id in range(X_train.shape[0]):
    for x in range(map_size):
        for y in range(map_size):
            node_id = x * map_size + y
            number_of_outgoing_edges = 0
            
            for dx, dy in directions:
                neighbor_x = x + dx
                neighbor_y = y + dy

                # Check if neighbor exists
                if 0 <= neighbor_x < map_size and 0 <= neighbor_y < map_size:                    
                    if dx == 1 and dy == 0:
                        edge_type = "DownLeft"
                    elif dx == -1 and dy == 0:
                        edge_type = "UpRight"
                    elif dx == 0 and dy == 1:
                        edge_type = "DownRight"
                    elif dx == 0 and dy == -1:
                        edge_type = "UpLeft"
                    elif dx == 1 and dy == -1:
                        edge_type = "Left"
                    elif dx == -1 and dy == 1:
                        edge_type = "Right"
                    
                    neighbor_id = neighbor_x * map_size + neighbor_y
                    graphs_train.add_graph_node_edge(graph_id, node_id, neighbor_id, edge_type)
print("Added neighbor edges")


Finished preparing node configuration



KeyboardInterrupt



In [ ]:
print("Preparing to add training features")
for graph_id in range(X_train.shape[0]):
    for x in range(map_size):
        for y in range(map_size):
            node_id = x * map_size + y

            # Brick type
            if X_train[graph_id][y][x] == -1:
                brick = 'A'
            elif X_train[graph_id][y][x] == 1:
                brick = 'B'
            else:
                brick = 'O'
            graphs_train.add_graph_node_property(graph_id, node_id, brick)

            # Brick Position
            graphs_train.add_graph_node_property(graph_id, node_id, f"x:{x}")
            graphs_train.add_graph_node_property(graph_id, node_id, f"y:{y}")

            # Neighbors
            # countA, countB, countO = 0, 0, 0
            # for dx, dy in directions:
            #     nx = x + dx
            #     ny = y + dy
            #     if 0 <= nx < map_size and 0 <= ny < map_size:
            #         neighbor_val = X_train[graph_id][ny][nx]
            #         if neighbor_val == -1:
            #             countA += 1
            #         elif neighbor_val == 1:
            #             countB += 1
            #         else:
            #             countO += 1
            # graphs_train.add_graph_node_property(graph_id, node_id, f"neighbors_A:{countA}")
            # graphs_train.add_graph_node_property(graph_id, node_id, f"neighbors_B:{countB}")
            # graphs_train.add_graph_node_property(graph_id, node_id, f"neighbors_O:{countO}")

            # What neighbor is in what direction
            for (dx, dy) in directions:
                nx = x + dx
                ny = y + dy
                dir_label = direction_labels[(dx, dy)]

                if 0 <= nx < map_size and 0 <= ny < map_size:
                    neighbor_val = X_train[graph_id][ny][nx]
                    if neighbor_val == -1:
                        neighbor_brick = 'A'
                    elif neighbor_val == 1:
                        neighbor_brick = 'B'
                    else:
                        neighbor_brick = 'O'

                    # Add property: e.g. "left:A" or "down_right:B"
                    graphs_train.add_graph_node_property(graph_id, node_id, f"{dir_label}:{neighbor_brick}")
                else:
                    # Optionally, indicate no neighbor in that direction
                    # graphs_train.add_graph_node_property(graph_id, node_id, f"{dir_label}:None")
                    pass

            # Is corner or border
            corner_flag = "True" if is_corner(x, y, map_size) else "False"
            border_flag = "True" if is_border(x, y, map_size) else "False"
            graphs_train.add_graph_node_property(graph_id, node_id, f"is_corner:{corner_flag}")
            graphs_train.add_graph_node_property(graph_id, node_id, f"is_border:{border_flag}")
            
            # # Distance to center
            # dist_center = manhattan_dist_to_center(x, y)
            # graphs_train.add_graph_node_property(graph_id, node_id, f"dist_center:{dist_center}")
            
            # # Odd or even
            # graphs_train.add_graph_node_property(graph_id, node_id, f"parity_x:{parity_label(x)}")
            # graphs_train.add_graph_node_property(graph_id, node_id, f"parity_y:{parity_label(y)}")

graphs_train.encode()
print("All training node features added.")


In [ ]:
print("Creating testing data")

# Create test graph
graphs_test = Graphs(
    X_test.shape[0],
    symbols=symbols,
    hypervector_size=args.hypervector_size,
    hypervector_bits=args.hypervector_bits,
    init_with = graphs_train
)

for graph_id in range(X_test.shape[0]):
    graphs_test.set_number_of_graph_nodes(graph_id, number_of_nodes)
    
graphs_test.prepare_node_configuration()
print("Finished preparing node configuration")

# Adding test nodes
for graph_id in range(X_test.shape[0]):
    for x in range(map_size):
        for y in range(map_size):
            node_id = x * map_size + y
            number_of_outgoing_edges = 0
            
            for dx, dy in directions:
                neighbor_x = x + dx
                neighbor_y = y + dy
                
                # Check if neighbor exists within bounds
                if 0 <= neighbor_x < map_size and 0 <= neighbor_y < map_size:
                    number_of_outgoing_edges += 1

            # Use the node_counter to create a unique label for each node
            graphs_test.add_graph_node(graph_id, node_id, number_of_outgoing_edges)

print("All nodes are acounted for")
graphs_test.prepare_edge_configuration()
edge_type = "Plain"

# Adding edges to the test graph
print("Started adding test edges")
for graph_id in range(X_test.shape[0]):
    for x in range(map_size):
        for y in range(map_size):
            node_id = x * map_size + y
            number_of_outgoing_edges = 0
    
            
            for dx, dy in directions:
                neighbor_x = x + dx
                neighbor_y = y + dy

                # Check if neighbor exists within bounds
                if 0 <= neighbor_x < map_size and 0 <= neighbor_y < map_size:

                    if dx == 1 and dy == 0:
                        edge_type = "DownLeft"
                    elif dx == -1 and dy == 0:
                        edge_type = "UpRight"
                    elif dx == 0 and dy == 1:
                        edge_type = "DownRight"
                    elif dx == 0 and dy == -1:
                        edge_type = "UpLeft"
                    elif dx == 1 and dy == -1:
                        edge_type = "Left"
                    elif dx == -1 and dy == 1:
                        edge_type = "Right"
                    
                    neighbor_id = neighbor_x * map_size + neighbor_y
                    graphs_test.add_graph_node_edge(graph_id, node_id, neighbor_id, edge_type)
print("Added neighbor edges")



In [ ]:
print("Preparing to add bricks (testing) + extra features...")
for graph_id in range(X_test.shape[0]):
    for x in range(map_size):
        for y in range(map_size):
            node_id = x * map_size + y
            
            # Brick Type
            if X_test[graph_id][y][x] == -1:
                brick = 'A'
            elif X_test[graph_id][y][x] == 1:
                brick = 'B'
            else:
                brick = 'O'
            graphs_test.add_graph_node_property(graph_id, node_id, brick)

            # Position
            graphs_test.add_graph_node_property(graph_id, node_id, f"x:{x}")
            graphs_test.add_graph_node_property(graph_id, node_id, f"y:{y}")

            # Neighbors
            # countA, countB, countO = 0, 0, 0
            # for dx, dy in directions:
            #     nx = x + dx
            #     ny = y + dy
            #     if 0 <= nx < map_size and 0 <= ny < map_size:
            #         neighbor_val = X_test[graph_id][ny][nx]
            #         if neighbor_val == -1:
            #             countA += 1
            #         elif neighbor_val == 1:
            #             countB += 1
            #         else:
            #             countO += 1
            # graphs_test.add_graph_node_property(graph_id, node_id, f"neighbors_A:{countA}")
            # graphs_test.add_graph_node_property(graph_id, node_id, f"neighbors_B:{countB}")
            # graphs_test.add_graph_node_property(graph_id, node_id, f"neighbors_O:{countO}")

            # neighbors and direction
            for (dx, dy) in directions:
                dir_label = direction_labels[(dx, dy)]
                nx = x + dx
                ny = y + dy
                if 0 <= nx < map_size and 0 <= ny < map_size:
                    neighbor_val = X_test[graph_id][ny][nx]
                    if neighbor_val == -1:
                        neighbor_brick = 'A'
                    elif neighbor_val == 1:
                        neighbor_brick = 'B'
                    else:
                        neighbor_brick = 'O'
                    graphs_test.add_graph_node_property(graph_id, node_id, f"{dir_label}:{neighbor_brick}")
                else:
                    # If you want to explicitly encode no neighbor:
                    # graphs_test.add_graph_node_property(graph_id, node_id, f"{dir_label}:None")
                    pass
                    
            # Border and Corner 
            corner_flag = "True" if is_corner(x, y, map_size) else "False"
            border_flag = "True" if is_border(x, y, map_size) else "False"
            graphs_test.add_graph_node_property(graph_id, node_id, f"is_corner:{corner_flag}")
            graphs_test.add_graph_node_property(graph_id, node_id, f"is_border:{border_flag}")

            # # Distance to center
            # dist_center = manhattan_dist_to_center(x, y)
            # graphs_test.add_graph_node_property(graph_id, node_id, f"dist_center:{dist_center}")

            # # Odd or even
            # graphs_test.add_graph_node_property(graph_id, node_id, f"parity_x:{parity_label(x)}")
            # graphs_test.add_graph_node_property(graph_id, node_id, f"parity_y:{parity_label(y)}")

graphs_test.encode()
print("All testing node features added.")

In [ ]:
# For plotting the epochs and results
train_accuracy_history = []
test_accuracy_history = []
f1_score_history = []

# The training of the GTM seems to effected by the random initialization,
# init more than one time and use the best one, often reduces training by hours
tm = None
best_score = 0
iterations = 0
n_tries = 1

In [ ]:
import time
from sklearn.metrics import f1_score

while (best_score < 85 or iterations < 5) and iterations < n_tries:
    tm1 = MultiClassGraphTsetlinMachine(
        args.number_of_clauses,
        args.T,
        args.s,
        depth=args.depth,
        message_size=args.message_size,
        message_bits=args.message_bits,
        max_included_literals=args.max_included_literals
    )
    
    start_training = time.time()
    tm1.fit(graphs_train, Y_train, epochs=1, incremental=True)
    stop_training = time.time()
        
    start_testing = time.time()
    y_pred_test = tm1.predict(graphs_test)
    stop_testing = time.time()
    
    y_pred_train = tm1.predict(graphs_train)
        
        # Calculate metrics
    result_test = 100 * (y_pred_test == Y_test).mean()
    result_train = 100 * (y_pred_train == Y_train).mean()
    f1 = f1_score(Y_test, y_pred_test, average='macro')
        
        # Store metrics
    test_accuracy_history.append(result_test)
    train_accuracy_history.append(result_train)
    f1_score_history.append(f1)

    if result_test > best_score:
        best_score = result_test
        tm = tm1
    
    print(
        # f"Epoch {i} | "
        f"Train Accuracy: {result_train:.2f} | "
        f"Test Accuracy: {result_test:.2f} | "
        f"F1 Score: {f1:.4f} | "
        f"Train Time: {stop_training - start_training:.2f} s | "
        f"Test Time: {stop_testing - start_testing:.2f} s"
    )
    iterations += 1

if not tm:
    print("No Good results")


In [ ]:
# tm = MultiClassGraphTsetlinMachine(
#     args.number_of_clauses,
#     args.T,
#     args.s,
#     depth=args.depth,
#     message_size=args.message_size,
#     message_bits=args.message_bits,
#     max_included_literals=args.max_included_literals
# )

for i in range(args.epochs):
    start_training = time.time()
    tm.fit(graphs_train, Y_train, epochs=1, incremental=True)
    stop_training = time.time()
    
    start_testing = time.time()
    y_pred_test = tm.predict(graphs_test)
    stop_testing = time.time()

    y_pred_train = tm.predict(graphs_train)
    
    # Calculate metrics
    result_test = 100 * (y_pred_test == Y_test).mean()
    result_train = 100 * (y_pred_train == Y_train).mean()
    f1 = f1_score(Y_test, y_pred_test, average='macro')
    
    # Store metrics
    test_accuracy_history.append(result_test)
    train_accuracy_history.append(result_train)
    f1_score_history.append(f1)
    
    print(
        f"Epoch {i} | "
        f"Train Accuracy: {result_train:.3f} | "
        f"Test Accuracy: {result_test:.3f} | "
        f"F1 Score: {f1:.4f} | "
        f"Train Time: {stop_training - start_training:.2f} s | "
        f"Test Time: {stop_testing - start_testing:.2f} s"
    )

    if result_train == 100 and result_test == 100:
        break



In [ ]:
import matplotlib.pyplot as plt

epochs_range = range(len(train_accuracy_history))
stop_n = -1
plt.figure(figsize=(10, 6))
plt.plot(epochs_range, train_accuracy_history, label='Train Accuracy')
plt.plot(epochs_range, test_accuracy_history, label='Test Accuracy')
#plt.plot(epochs_range, f1_score_history, label='F1 Score', marker='^')

plt.title("Tsetlin Machine Training Performance")
plt.xlabel("Epoch")
plt.ylabel("Metric Value")
plt.legend()
plt.grid(True)
# plt.savefig(f"Results/{dim}x{dim}_{args.number_of_clauses}c_{args.depth}depth_s{args.depth}_128hs_1hb_100%.pdf")
plt.savefig(f"Results/{dim}x{dim}_{args.number_of_clauses // 1_000}kC_{args.depth}d_{args.s}s_{max(test_accuracy_history):.1f}%.pdf")
plt.show()

    # args.number_of_clauses,
    # args.T,
    # args.s,
    # depth=args.depth,
    # message_size=args.message_size,
    # message_bits=args.message_bits,
    # max_included_literals=args.max_included_literals

In [ ]:
max_len = min([len(test_accuracy_history_20), len(test_accuracy_history_50), len(test_accuracy_history_80)])
epochs_ = np.arange(max_len)
plt.figure(figsize=(10, 6))
plt.plot(epochs_, test_accuracy_history_20[:max_len], label='20% Train')
plt.plot(epochs_, test_accuracy_history_50[:max_len], label='50% Train')
plt.plot(epochs_, test_accuracy_history_80[:max_len], label='80% Train')
plt.ylabel("Test Accuracy")
plt.xlabel("Epochs")
plt.legend()
plt.grid(True)
plt.savefig("Results/ComparingTrain%.pdf")
plt.show()

In [ ]:
np.save("Results/11x11_4_before", test_accuracy_history)

In [ ]:
before_0 = np.load("Results/11x11_0_before.npy")
before_2 = np.load("Results/11x11_2_before.npy")
before_4 = np.load("Results/11x11_4_before.npy")
max_len = min([len(before_0), len(before_2), len(before_4)])
epochs_range = np.arange(max_len)

plt.figure(figsize=(10, 6))
# plt.plot(epochs_range, train_accuracy_history, label='Train Accuracy', marker='o')
# plt.plot(epochs_range, test_accuracy_history, label='Test Accuracy', marker='s')
plt.plot(epochs_range, before_0[:max_len], label='Complete Game')
plt.plot(epochs_range, before_2[:max_len], label='2 Moves Before')
plt.plot(epochs_range, before_4[:max_len], label='4 Moves Before')

plt.title("Comparison")
plt.xlabel("Epoch")
plt.ylabel("Test Accuracy")
plt.legend()
plt.grid(True)
plt.savefig("Results/Comparison.pdf")
plt.show()

In [ ]:
import time
from tqdm import tqdm

tm1d = MultiClassGraphTsetlinMachine(
        args.number_of_clauses,
        args.T,
        args.s,
        depth=1,
        message_size=args.message_size,
        message_bits=args.message_bits,
        max_included_literals=args.max_included_literals
    )
tm2d = MultiClassGraphTsetlinMachine(
        args.number_of_clauses,
        args.T,
        args.s,
        depth=2,
        message_size=args.message_size,
        message_bits=args.message_bits,
        max_included_literals=args.max_included_literals
    )
tm3d = MultiClassGraphTsetlinMachine(
        args.number_of_clauses,
        args.T,
        args.s,
        depth=3,
        message_size=args.message_size,
        message_bits=args.message_bits,
        max_included_literals=args.max_included_literals
    )
tm5d = MultiClassGraphTsetlinMachine(
        args.number_of_clauses,
        args.T,
        args.s,
        depth=5,
        message_size=args.message_size,
        message_bits=args.message_bits,
        max_included_literals=args.max_included_literals
    )

test_accuracy_history_1d = []
test_accuracy_history_2d = []
test_accuracy_history_3d = []
test_accuracy_history_5d = []

training_time_1d = []
training_time_2d = []
training_time_3d = []
training_time_5d = []

# Training 1-depth model
print("Starting 1-depth model")
for i in tqdm(range(60)):
    start_time = time.time()
    tm1d.fit(graphs_train, Y_train, epochs=1, incremental=True)
    end_time = time.time()
    training_time_1d.append(end_time - start_time)

    y_pred_test = tm1d.predict(graphs_test)
    result_test = 100 * (y_pred_test == Y_test).mean()
    test_accuracy_history_1d.append(result_test)

# Training 2-depth model
print("Starting 2-depth model")
for i in tqdm(range(60)):
    start_time = time.time()
    tm2d.fit(graphs_train, Y_train, epochs=1, incremental=True)
    end_time = time.time()
    training_time_2d.append(end_time - start_time)

    y_pred_test = tm2d.predict(graphs_test)
    result_test = 100 * (y_pred_test == Y_test).mean()
    test_accuracy_history_2d.append(result_test)

# Training 3-depth model
print("Starting 3-depth model")
for i in tqdm(range(60)):
    start_time = time.time()
    tm3d.fit(graphs_train, Y_train, epochs=1, incremental=True)
    end_time = time.time()
    training_time_3d.append(end_time - start_time)

    y_pred_test = tm3d.predict(graphs_test)
    result_test = 100 * (y_pred_test == Y_test).mean()
    test_accuracy_history_3d.append(result_test)

# Training 5-depth model
print("Starting 5-depth model")
for i in tqdm(range(60)):
    start_time = time.time()
    tm5d.fit(graphs_train, Y_train, epochs=1, incremental=True)
    end_time = time.time()
    training_time_5d.append(end_time - start_time)

    y_pred_test = tm5d.predict(graphs_test)
    result_test = 100 * (y_pred_test == Y_test).mean()
    test_accuracy_history_5d.append(result_test)


epochs = len(test_accuracy_history_1d)


In [ ]:
import matplotlib.pyplot as plt

print(len(test_accuracy_history_1d))

epochs = np.arange(60)

plt.figure(figsize=(10, 6))
plt.plot(epochs, test_accuracy_history_1d, label='Depth=1')
plt.plot(epochs, test_accuracy_history_2d, label='Depth=2')
plt.plot(epochs, test_accuracy_history_3d, label='Depth=3')
plt.plot(epochs, test_accuracy_history_5d, label='Depth=5')
#plt.plot(epochs_range, f1_score_history, label='F1 Score', marker='^')

plt.title("Test Accuracy, depth comparison")
plt.xlabel("Epoch")
plt.ylabel("Test Accuracy")
plt.legend()
plt.grid(True)
# plt.savefig(f"Results/{dim}x{dim}_{args.number_of_clauses}c_{args.depth}depth_s{args.depth}_128hs_1hb_100%.pdf")
plt.savefig("Results/Comparison.pdf")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.plot(epochs, training_time_1d, label='Depth=1')
plt.plot(epochs, training_time_2d, label='Depth=2')
plt.plot(epochs, training_time_3d, label='Depth=3')
plt.plot(epochs, training_time_5d, label='Depth=5')
#plt.plot(epochs_range, f1_score_history, label='F1 Score', marker='^')

plt.title("Training time, depth comparison")
plt.xlabel("Epoch")
plt.ylabel("Time (s)")
plt.legend()
plt.grid(True)
# plt.savefig(f"Results/{dim}x{dim}_{args.number_of_clauses}c_{args.depth}depth_s{args.depth}_128hs_1hb_100%.pdf")
plt.savefig(f"Results/ComparisonTime.pdf")
plt.show()




In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Compute cumulative training time
cumulative_time_1d = np.cumsum(training_time_1d)
cumulative_time_2d = np.cumsum(training_time_2d)
cumulative_time_3d = np.cumsum(training_time_3d)
cumulative_time_5d = np.cumsum(training_time_5d)

plt.figure(figsize=(10, 6))

plt.plot(epochs+1, cumulative_time_1d, label='Depth=1')
plt.plot(epochs+1, cumulative_time_2d, label='Depth=2')
plt.plot(epochs+1, cumulative_time_3d, label='Depth=3')
plt.plot(epochs+1, cumulative_time_5d, label='Depth=5')

plt.title("Cumulative Training Time, Depth Comparison")
plt.xlabel("Epoch")
plt.ylabel("Cumulative Time (s)")
plt.legend()
plt.grid(True)
plt.savefig(f"Results/ComparisonTime_cumulative.pdf")
plt.show()
